# MITx (Mid-lat ionospheric trough)

> Abstract: Access to the `MITx_LP` and `MITxTEC` products from the PRISM project. These give information about the boundaries and minima of the midlatitude ionospheric trough derived from the Langmuir Probe (LP) measurements and Total Electron Content (TEC) respectively.

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import SwarmRequest

## PRISM product information

The [PRISM project](https://www.gfz-potsdam.de/en/section/geomagnetism/projects/prism-plasmapause-related-boundaries-in-the-topside-ionosphere-as-derived-from-swarm-measurements/) (*Plasmapause Related boundaries in the topside Ionosphere as derived from Swarm Measurements*) has two data product outputs derived from Swarm which provide the minima of the mid-latitude ionospheric trough (MIT):  
`MITx_LP`: derived from the Langmuir Probe (LP)  
`MITxTEC`: derived from the GPS Total Electron Content (TEC)  

These are implemented in VirES as four collection types:  
`SW_OPER_MITx_LP_2F`  
`SW_OPER_MITx_LP_2F:ID`  
`SW_OPER_MITxTEC_2F`  
`SW_OPER_MITxTEC_2F:ID`  
where `x` = `A/B/C` for each spacecraft.

The base collections (`SW_OPER_MITx_LP_2F` and `SW_OPER_MITxTEC_2F`) contain information on the positions of the MIT minima. The sub-collections with `:ID` appended contain information on the boundaries.

Similarly, the midnight plasmapause index (PPI) is provided in the collections:  
`SW_OPER_PPIxFAC_2F`  
`SW_OPER_PPIxFAC_2F:ID`

For further details about the implementation in VirES, see:
- https://nbviewer.jupyter.org/github/pacesm/jupyter_notebooks/blob/master/PRISM/PRISM_00_data_access.ipynb

## MIT_LP example

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MITA_LP_2F")
request.set_products(
    measurements=[
        "Counter", "Latitude_QD", "Longitude_QD", "MLT_QD", "L_value", "SZA",
        "Ne", "Te", "Depth", "DR", "Width", "dL", "PW_Gradient", "EW_Gradient",
        "Quality"
    ],
)

data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
    asynchronous=False, show_progress=False
)
ds = data.as_xarray()

In [ ]:
ds

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MITA_LP_2F:ID")
request.set_products(
    measurements=SwarmRequest.PRODUCT_VARIABLES["MIT_LP:ID"], # request all dataset variables
    auxiliaries=['QDLat', 'QDLon', 'MLT'], # QD coordinates and MLT calculated by VirES
)

data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
    asynchronous=False, show_progress=False
)
ds_id = data.as_xarray()

In [ ]:
ds_id

## MIT_TEC example

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_MITATEC_2F")
request.set_products(
    measurements=[
        "Counter", "Latitude_QD", "Longitude_QD", "MLT_QD", "L_value", "SZA",
        "TEC", "Depth", "DR", "Width", "dL", "PW_Gradient", "EW_Gradient",
        "Quality",
    ],
)

data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
    asynchronous=False, show_progress=False
)
ds = data.as_xarray()
ds

## PPI_FAC example

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_PPIAFAC_2F")
request.set_products(
    measurements=[
        "Counter", "Latitude_QD", "Longitude_QD", "MLT_QD", "L_value", "SZA", "Sigma", "PPI", "dL", "Quality"
    ],
)
data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
    asynchronous=False, show_progress=False
)
ds = data.as_xarray()
ds

In [ ]:
request = SwarmRequest()
request.set_collection("SW_OPER_PPIAFAC_2F:ID")
request.set_products(
    measurements=[
        "Counter", "Latitude_QD", "Longitude_QD", "MLT_QD", "L_value", "SZA", "Position_Quality", "PointType"
    ],
)
data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
    asynchronous=False, show_progress=False
)
ds = data.as_xarray()
ds